# Content-Based

In [56]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from nltk import  PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import spacy

In [57]:
!gdown 'https://drive.google.com/uc?id=1-xdr4LtRuJETLfVzImRLVhTRTzxPwLFO'
movies = pd.read_csv('tmdb_5000_movies.csv')

!gdown 'https://drive.google.com/uc?id=1WG7MQTFYnK2BCdLohepuc1AlfRrE1cly'
credits = pd.read_csv('tmdb_5000_credits.csv')

Downloading...
From: https://drive.google.com/uc?id=1-xdr4LtRuJETLfVzImRLVhTRTzxPwLFO
To: /content/tmdb_5000_movies.csv
100% 5.70M/5.70M [00:00<00:00, 221MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WG7MQTFYnK2BCdLohepuc1AlfRrE1cly
To: /content/tmdb_5000_credits.csv
100% 40.0M/40.0M [00:00<00:00, 124MB/s]


In [58]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [59]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [60]:
movies.merge(credits, left_on='id', right_on='movie_id')
movies = movies.merge(credits, left_on='id', right_on='movie_id')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [61]:
# Relevent data
# genre, id, keywords, title, overview, cast, crew
movies = movies[['id', 'genres', 'title_x', 'overview', 'keywords', 'cast', 'crew']]
movies.head()

,id,genres,title_x,overview,keywords,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [62]:
movies.isnull().sum();
movies.duplicated().sum();

In [63]:
movies.dropna(inplace = True)

In [64]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [65]:
# Convert list of dictinory into single list for genres, cast and keywords
def convert(obj):
    new_list = []
    for i in ast.literal_eval(obj):
        new_list.append(i['name'])
    return new_list

In [66]:
# Convert list of dictinory into single list for crew -> on the basis of director
def convert1(obj):
    new_list = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            new_list.append(i['name'])
            break
    return new_list

In [67]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['crew'] = movies['crew'].apply(convert1)

In [68]:
movies.head()

,id,genres,title_x,overview,keywords,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]",Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,"[Action, Adventure, Crime]",Spectre,A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,"[Action, Adventure, Science Fiction]",John Carter,"John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [69]:
movies.iloc[4].overview

"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands."

In [70]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [71]:
# Revoming spcace
movies['genres'] = movies['genres'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x :[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x :[i.replace(" ", "") for i in x])

In [72]:
movies.head()

,id,genres,title_x,overview,keywords,cast,crew
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,"[Action, Adventure, Crime]",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,"[Action, Adventure, ScienceFiction]",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [73]:
movies['tag'] = movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew']

In [74]:
movies.head()

,id,genres,title_x,overview,keywords,cast,crew,tag
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,"[Action, Adventure, Crime]",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,"[Action, Adventure, ScienceFiction]",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [75]:
new_df = movies[['id', 'title_x', 'tag']]

In [76]:
# List -> String | tag
new_df.loc[:, 'tag'] = new_df['tag'].apply(lambda x: " ".join(x))
new_df.head()

,id,title_x,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [77]:
new_df.loc[:, 'tag'] = new_df['tag'].apply(lambda x: x.lower())

In [78]:
new_df.head()

,id,title_x,tag
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [79]:
# Stemming
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [80]:
new_df.loc[:,'tag'] = new_df['tag'].apply(stem)

In [81]:
stop_words = set(stopwords.words('english'))

# Add custom stopwords
custom_stopwords = ['want', 'watch', 'movie', 'something', 'film', 'recommend', 'looking', 'like', 'genre', 'based', 'good', 'bad', 'best', 'worst', 'interesting', 'suggest', 'kind', 'type', 'series', 'please', 'find', 'see', 'can', 'tell', 'give', 'suggestion', 'help', 'think', 'anything', 'similar', 'available', 'latest', 'old', 'focus', 'has', 'feel']

# Function to get the stem of a word
def stem_word(word):
    return ps.stem(word)

# Create a set of stemmed stop words
stemmed_stop_words = set(stem_word(word) for word in custom_stopwords)

# Update the stop words list with stemmed versions
stop_words.update(stemmed_stop_words)

def remove_stopwords(text):
    return " ".join([word for word in text.split() if ps.stem(word) not in stop_words])

In [82]:
new_df['tag'] = new_df['tag'].apply(remove_stopwords)

<ipython-input-82-aa6b0254e7f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(remove_stopwords)


In [83]:
# Text vectorization + Removing stop words
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tag']).toarray()   # Returns scipy sparse matrix hence convering to numpy array

In [84]:
# Most used words
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [85]:
# Initilize cosine_similarity
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.        , 0.02795085, ..., 0.02357023, 0.        ,
        0.        ],
       [0.        , 1.        , 0.02760788, ..., 0.02328101, 0.        ,
        0.        ],
       [0.02795085, 0.02760788, 1.        , ..., 0.02635231, 0.        ,
        0.        ],
       ...,
       [0.02357023, 0.02328101, 0.02635231, ..., 1.        , 0.02300219,
        0.02300219],
       [0.        , 0.        , 0.        , ..., 0.02300219, 1.        ,
        0.0952381 ],
       [0.        , 0.        , 0.        , ..., 0.02300219, 0.0952381 ,
        1.        ]])

In [86]:
similarity[0][2]  # similarity between 0 and 2 movie

0.02795084971874737

In [87]:
# Recomendation fuction
def recommend_title(name):
    m_id = new_df[new_df['title_x'] == name].index[0]

    # List of similarity scores between the m_id and all other movies.
    distances = similarity[m_id]

    # Sorts the movies based on their similarity scores, excluding the selected movie
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:11]
    for i in movies_list:
        print(new_df.iloc[i[0]].title_x)

In [88]:
recommend_title('Spectre')

Skyfall
Quantum of Solace
Never Say Never Again
Safe Haven
From Russia with Love
Dr. No
Restless
Die Another Day
Octopussy
Thunderball


In [89]:
recommend_title('The Dark Knight Rises')

The Dark Knight
Batman Returns
Batman Begins
Batman Forever
Batman
Batman & Robin
Batman: The Dark Knight Returns, Part 2
Teenage Mutant Ninja Turtles
Batman v Superman: Dawn of Justice
Slow Burn


In [90]:
# Process the sentence and get the key words
def preprocess_sentence(sentence):
    # Tokenize and lowercase the sentence
    sentence = sentence.lower()

    # Stem the remaining words
    processed_sentence = stem(sentence)

    # Remove stop words
    processed_sentence = remove_stopwords(processed_sentence)

    # Split the sentence into words to print each word
    for word in processed_sentence.split():
        print(word)

    return processed_sentence

In [91]:
# Extract the names of crew or cast or any date

nlp = spacy.load("en_core_web_sm")
def extract_named_entities(sentence):
    doc = nlp(sentence)
    return [token.text.lower() for token in doc.ents]

In [92]:
# For First Name and Last name
def normalize_text(text):
    # Convert to lowercase and remove spaces
    return text.lower().replace(" ", "")

In [93]:
# For First Name and Last name
def combine_named_entities(named_entities):
    return ' '.join(named_entities).strip()

In [94]:
# Recommendation for named entities

def recommend_named_entities(sentence):
    # Extract named entities from the input sentence
    named_entities = extract_named_entities(sentence)

    # Normalize named entities for matching
    normalized_entities = [normalize_text(entity) for entity in named_entities]

    # Create a DataFrame to store the relevance scores of each movie
    relevance_scores = []

    for index, row in new_df.iterrows():
        # Check if any named entity is in the movie tag
        if any(entity in row['tag'] for entity in normalized_entities):
            relevance_scores.append((index, row['title_x']))

    # Sort the movies based on their relevance scores
    relevance_scores.sort(key=lambda x: x[0])

    print("\nRecommended movies based on named entities\n")
    for _, title in relevance_scores[:10]:
        print(title)

In [95]:
# Recomendation based on description

def recommend_sentence(sentence):

    # preprocess
    processed_sentence = preprocess_sentence(sentence)

    # Vectorize the input sentence
    sentence_vector = cv.transform([processed_sentence]).toarray()

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(sentence_vector, vectors).flatten()

    # Get top 10 similar movies
    top_indices = similarity_scores.argsort()[::-1][1:11]

    print("\nRecommended movies based on your description\n")
    for i in top_indices:
        print(new_df.iloc[i]['title_x'])

In [96]:
def recommend(sentence):
    # Extract named entities from the input sentence
    named_entities = extract_named_entities(sentence)

    if named_entities:
        print("Named entities found: ", named_entities)
        recommend_named_entities(sentence)
    else:
        print("Using text-based recommendation.")
        recommend_sentence(sentence)

In [97]:
recommend('I want to watch something supernatural and horror with suspense')

Using text-based recommendation.
supernatur
horror
suspens

Recommended movies based on your description

Oculus
Dylan Dog: Dead of Night
White Noise
They
Jennifer's Body
The Horror Network Vol. 1
Silent House
It Follows
Open Secret
R.L. Stine's Monsterville: The Cabinet of Souls


In [98]:
recommend('I want to watch batman movie')

Using text-based recommendation.
batman

Recommended movies based on your description

Batman: The Dark Knight Returns, Part 2
The Dark Knight
The Dark Knight Rises
Batman Forever
The Lego Movie
Batman & Robin
Batman Begins
Batman v Superman: Dawn of Justice
Southpaw
Drag Me to Hell


In [99]:
recommend('acient and historic')

Using text-based recommendation.
acient
histor

Recommended movies based on your description

The Other Conquest
Saints and Soldiers
The Great Debaters
Agora
Woodstock
New Nightmare
One Night with the King
Anne of Green Gables
Time Bandits
Cleopatra


In [100]:
recommend('I want to watch an acient and historic movie')

Using text-based recommendation.
acient
histor

Recommended movies based on your description

The Other Conquest
Saints and Soldiers
The Great Debaters
Agora
Woodstock
New Nightmare
One Night with the King
Anne of Green Gables
Time Bandits
Cleopatra


In [101]:
recommend('Can you recommend a film that focus on ancient history and has a historic feel')

Using text-based recommendation.
ancient
histori
histor

Recommended movies based on your description

The Lords of Salem
Agora
Bulletproof Monk
Time Bandits
Reign of Assassins
The Conspirator
Bill & Ted's Excellent Adventure
Night at the Museum
Subconscious
Nomad: The Warrior


In [102]:
recommend('Can you recommend movies of JohnnyDepp')

Named entities found:  ['johnnydepp']

Recommended movies based on named entities

Pirates of the Caribbean: At World's End
Pirates of the Caribbean: Dead Man's Chest
The Lone Ranger
Pirates of the Caribbean: On Stranger Tides
Alice in Wonderland
Alice Through the Looking Glass
Charlie and the Chocolate Factory
Dark Shadows
Rango
Pirates of the Caribbean: The Curse of the Black Pearl


In [103]:
recommend('Can you recommend movies of Zoe Saldana')

Named entities found:  ['zoe saldana']

Recommended movies based on named entities

Avatar
Star Trek Into Darkness
Star Trek Beyond
Guardians of the Galaxy
Star Trek
Pirates of the Caribbean: The Curse of the Black Pearl
The Terminal
The Book of Life
Vantage Point
Colombiana
